In [1]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/NVIDIA/OpenSeq2Seq.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q --depth 1 {git_repo_url}
  !git checkout e958b7d
  !pip uninstall -y -q pymc3
  !pip install --upgrade joblib
  #!cd {project_name} && pip install -q -r requirements.txt
  !pip install -q youtube-dl librosa python_speech_features sentencepiece
  
  # create eval config
  !cp {project_name}/example_configs/speech2text/jasper10x5_LibriSpeech_nvgrad.py {project_name}/conf.py
  !sed -i -e 's/\/data\/librispeech\/librivox-test-clean/test/' {project_name}/conf.py
  #!sed -i -e 's/# "use_lang/"use_lang/' {project_name}/conf.py
  !echo 'backend = "librosa"' >> {project_name}/conf.py 
  #!cat {project_name}/conf.py
  !echo "wav_filename, wav_filesize, transcript" > {project_name}/test.csv
  !echo "test.wav, UNUSED, UNUSED" >> {project_name}/test.csv
  !pip install tensorflow-gpu==1.15
  
import sys
sys.path.append(project_name)
from IPython.display import YouTubeVideo

fatal: not a git repository (or any of the parent directories): .git
Requirement already up-to-date: joblib in /usr/local/lib/python3.6/dist-packages (0.15.1)
     |████████████████████████████████| 1.8MB 8.0MB/s 
     |████████████████████████████████| 1.1MB 52.6MB/s 
     |████████████████████████████████| 411.5MB 42kB/s 
     |████████████████████████████████| 3.8MB 44.1MB/s 
     |████████████████████████████████| 512kB 42.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7536ebe7a5aa765be1f7f4efddae7d94461b63954ae90accb2de00eaf492a966
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estima

In [2]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}
  
if not exists(join(project_name, 'w2l_log_folder')):
  download_from_google_drive('1gzGT8HoVNKY1i5HNQTKaSoCu7JHV4siR', 'jasper_10x5_dr_sp_nvgrad.zip')
  !unzip jasper_10x5_dr_sp_nvgrad.zip
  !mv checkpoint {project_name}/jasper_log_folder

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   1736      0 --:--:-- --:--:-- --:--:--  1728
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2960M    0 2960M    0     0  58.1M      0 --:--:--  0:00:50 --:--:-- 48.6M
Archive:  jasper_10x5_dr_sp_nvgrad.zip
   creating: checkpoint/
  inflating: checkpoint/checkpoint   
  inflating: checkpoint/model.ckpt-439200.data-00000-of-00001  
  inflating: checkpoint/model.ckpt-439200.index  
  inflating: checkpoint/model.ckpt-439200.meta  


In [3]:
!pip install unidecode
import torch
tacotron2 = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tacotron2')
!pip install numpy scipy librosa unidecode inflect librosa
import numpy as np
from scipy.io.wavfile import write

     |████████████████████████████████| 245kB 8.5MB/s 


Downloading: "https://github.com/nvidia/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip


In [4]:
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

Using cache found in /root/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub


WaveGlow(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0): WN(
      (in_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
        (1): Conv1d(51

In [5]:
def make_tts_list(text = "What can I do for yu?"):    
    sequence = np.array(tacotron2.text_to_sequence(text, ['english_cleaners']))[None, :]
    sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.int64)
    with torch.no_grad():
        _, mel, _, _ = tacotron2.infer(sequence)
        audio = waveglow.infer(mel)
    audio_numpy = audio[0].data.cpu().numpy()
    rate = 22050
    
    # Audio(audio_numpy, rate=rate)
    return audio_numpy

In [11]:
from IPython.display import Audio
rate = 22050

Reset Balance

In [35]:
balance = 5000
my_stock = [-1, 10, 10]
order_name = ['None', 'Buy', 'Sell']
stock_name = ['None', 'Amazon dot com', 'Microsoft']
stock_price = [-1, 2700, 196]

ASK to user

In [36]:
speak_to_user = make_tts_list("What can I do for yu")
Audio(speak_to_user, rate=rate)

Listen from user

In [37]:
print("Microphone is not available")
print("Input your audio file")
# speak_to_user = make_tts_list("Microphone is not available. Input your audio file")
# Audio(speak_to_user, rate=rate)
#1: buy amazon, 2: sell amazon, 3: buy microsoft, 4: sell microsoft
audio_input = input("1~4 : ")

Microphone is not available
Input your audio file
1~4 : 3


In [38]:
import IPython
order = f'{audio_input}.wav'
IPython.display.Audio(order, autoplay=True)

In [39]:
!ffmpeg -loglevel panic -y -i {order} -acodec pcm_s16le -ac 1 -ar 16000 {project_name}/test.wav
!cd {project_name} && python run.py --config_file conf.py --mode=infer --infer_output_file=output.txt --use_horovod=False --num_gpus=1 --batch_size_per_gpu 1












The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



*** Restoring from the latest checkpoint
*** Loading model from jasper_log_folder/model.ckpt-439200
*** Inference config:
{'batch_size_per_gpu': 1,
 'data_layer': <class 'open_seq2seq.data.speech2text.speech2text.Speech2TextDataLayer'>,
 'data_layer_params': {'backend': 'librosa',
                       'dataset_files': ['test.csv'],
                       'dither': 1e-05,
                       'input_type': 'logfbank',
                       'norm_per_feature': True,
                       'num_audio_features': 64,
                       'pad_to': 16,
                       'precompute_mel_basis': True,
          

In [40]:
f = open("OpenSeq2Seq/output.txt")
order_in_text = f.read(43)
order_in_text = f.read()
order_in_text = order_in_text.split(' ')
f.close()
print(order_in_text)

['i', 'microsopt\n']


In [41]:
#0:none 1:buy 2:sell, 0: None 1:Amazon, 2:Microsoft
processing_order = [0, 0]
if order_in_text[0][0]=='b' or order_in_text[0][0]=='i' :
  processing_order[0] = 1
elif order_in_text[0][0]=='s':
  processing_order[0] = 2
else:
  processing_order[0] = 0

if order_in_text[1][0]=='a':
  processing_order[1] = 1
elif order_in_text[1][0]=='m':
  processing_order[1] = 2
else:
  processing_order[0] = 0
  processing_order[1] = 0

In [43]:
if processing_order[0] ==0:
  print("Your order is not available")
else:
  print(f"Your order: {order_name[processing_order[0]]} {stock_name[processing_order[1]]}" )

Your order: Buy Microsoft


Process order

In [44]:
#0:none         1:buy          2:sell, 0:Amazon, 1:Microsoft
# processing_order[0] = 2 # 1 or 2
# processing_order[1] = 1 # 1 or 2
# balance = 100
# my_stock = [0,0]
text = ""
if processing_order[0] == 1:
  #Do buy process
  price = int(stock_price[processing_order[1]] *(np.random.rand(1)[0]/3+0.85))
  if balance >= price:
    balance -= price
    my_stock [processing_order[1]] += 1
    text = f"You bought {stock_name[processing_order[1]]} for {price} dollars"
  else:
    text = f"You can't, You have only {balance} dollars"

elif processing_order[0] == 2:
  #Do sell process
  price = int(stock_price[processing_order[1]] *(np.random.rand(1)[0]/3+0.85))
  if my_stock[processing_order[1]] >= 1:    
    my_stock [processing_order[1]] -= 1
    balance += price
    text = f"You sold {stock_name[processing_order[1]]} for {price} dollars"
  else:
    text = f"You can't, You have only {my_stock[processing_order[1]]} {stock_name[processing_order[1]]} shares"
else:
  #not understandable
  text = "I can't understand what you are saying"

print(text)
respond = make_tts_list(text)
Audio(respond, rate=rate, autoplay=True)

You bought Microsoft for 223 dollars


In [46]:
text = f"now, your balance is {balance} dollars, and you have {my_stock[1]} {stock_name[1]} shares and {my_stock[2]} {stock_name[2]} shares."
respond = make_tts_list(text)
print(text)
Audio(respond, rate=rate, autoplay=True)

now, your balance is 4777 dollars, and you have 10 Amazon dot com shares and 11 Microsoft shares.
